In [1]:
from utils import read_data
from utils import result
import torch
from torch import nn
from transformers import BertModel
import numpy as np
from torch.optim import Adam
from tqdm import tqdm
from transformers import BertTokenizer

In [ ]:
train_sentence, test_sentence, sarcasm_sentence, train_token, test_token, sarcasm_token, y_train, y_test, y_sarcasm = read_data()

The following code implementation uses this blog (URL: https://towardsdatascience.com/text-classification-with-bert-in-pytorch-887965e5820f) as reference.

In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
class BertClassifier(nn.Module):
    def __init__(self, dropout=0.5):
        super(BertClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 3)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        return self.relu(self.linear(self.dropout(output)))

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, tweets, labels):
        self.labels = labels
        self.texts = [tokenizer(tweet, padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for tweet in tweets]

    def classes(self):
        return self.labels

    def len(self):
        return len(self.labels)

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        return self.texts[idx]

    def __getitem__(self, idx):
        texts_batch = self.get_batch_texts(idx)
        y_batch = self.get_batch_labels(idx)
        return texts_batch, y_batch

In [ ]:
train_data = Dataset(train_sentence, list(y_train))
test_data = Dataset(test_sentence, list(y_test))
sarcasm_data = Dataset(sarcasm_sentence, list(y_sarcasm))

In [ ]:
def train(model, train_data, val_data, learning_rate, epochs):

    train, val = train_data, val_data

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=32, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=32)

    device = torch.device("cuda")
    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    model = model.cuda()
    criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()

                print(f'Train Loss: {batch_loss.item(): .3f} | Train Accuracy: {acc / 32: .3f} ')
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')
                  
EPOCHS = 5
model = BertClassifier()
LR = 1e-6
              
train(model, train_data, test_data, LR, EPOCHS)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  0%|          | 1/1491 [00:00<17:24,  1.43it/s]

Train Loss:  1.241 | Train Accuracy:  0.219 


  0%|          | 2/1491 [00:01<17:01,  1.46it/s]

Train Loss:  1.173 | Train Accuracy:  0.344 


  0%|          | 3/1491 [00:02<16:58,  1.46it/s]

...

Train Loss:  0.553 | Train Accuracy:  0.750 


  5%|▍         | 70/1491 [00:47<15:58,  1.48it/s]

Train Loss:  0.617 | Train Accuracy:  0.812 


  5%|▍         | 70/1491 [00:47<16:12,  1.46it/s]


KeyboardInterrupt: ignored

In [ ]:
device = torch.device("cuda")
pred_test = []
with torch.no_grad():
    val_dataloader = torch.utils.data.DataLoader(test_data, batch_size=32)
    for val_input, val_label in val_dataloader:

        val_label = val_label.to(device)
        mask = val_input['attention_mask'].to(device)
        input_id = val_input['input_ids'].squeeze(1).to(device)

        output = model(input_id, mask)
        pred_test.append(np.array(output.argmax(dim=1).tolist()))
        # acc = (output.argmax(dim=1) == val_label).sum().item()
        # total_acc_val += acc

In [ ]:
device = torch.device("cuda")
pred_sarcasm = []
with torch.no_grad():
    val_dataloader = torch.utils.data.DataLoader(sarcasm_data, batch_size=1)
    for val_input, val_label in val_dataloader:

        val_label = val_label.to(device)
        mask = val_input['attention_mask'].to(device)
        input_id = val_input['input_ids'].squeeze(1).to(device)

        output = model(input_id, mask)
        pred_sarcasm.append(output.argmax(dim=1).tolist())

In [ ]:
pred_sarcasm = np.array(pred_sarcasm).flatten()

In [ ]:
pred_test_final = []
for a in pred_test:
  for j in a:
    pred_test_final.append(j)

In [ ]:
pred_test_final = np.array(pred_test_final)
len(pred_test_final)

10050

In [ ]:
result(y_test, y_sarcasm, pred_test_final, pred_sarcasm, show=True)

Average Accuracy is 0.632636815920398
Average Recall is 0.6435551428093408
F1-Score is 0.610118837546262
Average Accuracy on sarcasm dataset is 0.5465116279069767
Average Recall on sarcasm dataset is 0.5933954933954934
F1-Score on sarcasm dataset is 0.5558441558441558


(0.632636815920398,
 0.6435551428093408,
 0.610118837546262,
 0.5465116279069767,
 0.5933954933954934,
 0.5558441558441558)

In [ ]:
len(pred_test)

315

In [ ]:
pred_test[0]

array([2, 0, 0, 0, 2, 2, 1, 0, 1, 2, 2, 0, 1, 2, 2, 1, 2, 2, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 2, 0, 1, 2])